# **1. Configurando a instância com GPU**
Vá nas opções Runtime > Change runtime type > GPU


# Configurando CUDA e Tensowflow

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__ )

In [0]:

if tf.__version__ !='1.12.2':
  !pip install --upgrade tensorflow-gpu==1.12.2
  !wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
  !dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
  !apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
  !apt-get update
  !apt-get install cuda=9.0.176-1
  !echo ****** Cuda reinstall completed. Restart runtime now! *******
  exit()

# **2. Abrindo uma instância SSH para acesso externo**
Ref: https://imadelhanafi.com/posts/google_colal_server/

Ref: https://ngrok.com/download

Iremos abrir um túnel SSH que permitirá a quem tiver a chave de acesso de se conectar à VM e executar os scripts...

In [0]:
# Gerando a senha que será utilizada para conectar o SSH
import random, string
password = 'INSIRA_SENHA_SSH'

In [0]:
# Baixando pacotes necessários para abrir um SSH
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

In [0]:
# Definindo a senha definada ao usuário ROOT
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "ClientAliveInterval 240" >> /etc/ssh/sshd_config
! echo "ClientAliveCountMax 2" >> /etc/ssh/sshd_config

! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

# Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

In [0]:
# Baixando o ngrok
!wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq -n ngrok-stable-linux-amd64.zip
!rm ngrok-stable-linux-amd64.zip

In [0]:
# Configurando AUTH Token
import getpass
print("Insira a seguir o AUTH Token disponivel em https://dashboard.ngrok.com/auth")
authtoken = getpass.getpass()

In [0]:
# Abrindo o Tunel 
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

In [0]:
# Exibindo os dados necessários para fazer a conexão SSH
print("Root password: {}".format(password))
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

# Exemplo de conexão ssh utilizando puTTY para testes
# putty -ssh root@0.tcp.ngrok.io -P portaAqui
# putty -ssh root@0.tcp.ngrok.io -P 19773

# Instância aberta 
A partir de agora, você pode utilizar conexões ssh para executar comandos nessa instância.

Lembre-se sempre que os arquivos da instância funcionam no diretório **/content**

# Fechando a Instância
Para fechar a instância e garantir que o SSH e seus dados do Google drive não vão ficar abertos indiscriminadamente é necessário resetar o runtime.

Para isso vá em Runtime > Reset all runtimes

Obs.: Reset != Restart

# **3. Integrando com a pasta do Google Drive**
Vamos criar link desta VM com a pasta desejada no Google Drive. 

Isso será necessário para que os scripts na pasta "dataday" sejam acessíveis pela SSH externo.

In [0]:
from google.colab import drive 
drive.mount('/content/gdrive') 

# **4. Baixando os Scripts e Models**

In [0]:
from pathlib import Path

In [0]:
! mkdir /content/morph

In [0]:
! cd /content/morph/

In [0]:
if not Path('stylegan-encoder').is_dir():
# !rm -rf sample_data
  !git clone https://github.com/pbaylies/stylegan-encoder
if not Path('InterFaceGAN').is_dir():
  !git clone https://github.com/tr1pzz/InterFaceGAN.git

In [0]:
###resnet do encoder
if not Path('stylegan-encoder/data/finetuned_resnet.h5').exists():
  !gdown https://drive.google.com/uc?id=1aT59NFy9-bNyXjDuZOTMl0qX0jmZc6Zb
  !mkdir stylegan-encoder/data
  !mv finetuned_resnet.h5 stylegan-encoder/data

In [0]:
###stylegan do interpolador
if not Path("InterFaceGAN/models/pretrain/karras2019stylegan-ffhq-1024x1024.pkl").exists():
  !gdown https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ
  !mv karras2019stylegan-ffhq-1024x1024.pkl InterFaceGAN/models/pretrain/karras2019stylegan-ffhq-1024x1024.pkl

# **5. Verificando funcionamento dos Scripts**
Após todas as etapas anteriores, é necessário copiar os scripts contidos em **/src/scripts** para **/content**. Dessa forma o collab poderá utilizar destes scripts para realizar as interpolações que serão exibidas no website.

In [0]:
# TESTANDO O PROCCESS.PY
'''
No Script proccess.py vão as os seguintes parâmetros:
- Caminho para as imagens ainda não processadas pela StyleGAN.
- Caminho para salvar as imagens interpoladas finalizadas.
- Pasta backup para salvar as imagens que já foram processadas.
'''

! cd /content
! python process.py /content/gdrive/My\ Drive/dataday/dataday_interpolar/website/nao_processadas /content/gdrive/My\ Drive/dataday/dataday_interpolar/website/static/interpolacoes /content/gdrive/My\ Drive/dataday/dataday_interpolar/website/processadas

In [0]:
# TESTANDO CHECK.PY
'''
O script check.py tem como objetivo verificar se proccess.py se encontra em execução, retornando:
- "finalizado" caso proccess.py não esteja em execução
- "processando" caso proccess.py esteja em execução.
'''
 ! cd /content
 ! python check.py